In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
import pandas as pd
import tensorflow
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Neural_Networks')

In [ ]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

dataset = pd.read_csv(r"forestfires.csv")


In [ ]:
dataset.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small


In [ ]:
dataset.month.replace(('jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'),(1,2,3,4,5,6,7,8,9,10,11,12), inplace=True)
dataset.day.replace(('mon','tue','wed','thu','fri','sat','sun'),(1,2,3,4,5,6,7), inplace=True)
dataset.size_category.replace(('small','large'),(0,1),inplace=True)

In [ ]:
dataset = dataset.astype({"FFMC":'int',"DMC":'int',"DC":'int',"ISI":'int',"temp":'int',"wind":'int',"rain":'int',"area":'int'})

In [ ]:
dataset.dtypes

month            int64
day              int64
FFMC             int64
DMC              int64
DC               int64
ISI              int64
temp             int64
RH               int64
wind             int64
rain             int64
area             int64
dayfri           int64
daymon           int64
daysat           int64
daysun           int64
daythu           int64
daytue           int64
daywed           int64
monthapr         int64
monthaug         int64
monthdec         int64
monthfeb         int64
monthjan         int64
monthjul         int64
monthjun         int64
monthmar         int64
monthmay         int64
monthnov         int64
monthoct         int64
monthsep         int64
size_category    int64
dtype: object

In [ ]:
df =dataset.drop(['dayfri','daymon','daysat','daysun','daythu','daytue','daywed','monthapr','monthaug','monthdec','monthfeb','monthjan','monthjul','monthjun','monthmar','monthmay','monthnov','monthoct','monthsep','size_category'],axis=1)

In [ ]:
df

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,3,5,86,26,94,5,8,51,6,0,0
1,10,2,90,35,669,6,18,33,0,0,0
2,10,6,90,43,686,6,14,33,1,0,0
3,3,5,91,33,77,9,8,97,4,0,0
4,3,7,89,51,102,9,11,99,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
512,8,7,81,56,665,1,27,32,2,0,6
513,8,7,81,56,665,1,21,71,5,0,54
514,8,7,81,56,665,1,21,70,6,0,11
515,8,6,94,146,614,11,25,42,4,0,0


In [ ]:
print("Head:", df.head())

Head:    month  day  FFMC  DMC   DC  ISI  temp  RH  wind  rain  area
0      3    5    86   26   94    5     8  51     6     0     0
1     10    2    90   35  669    6    18  33     0     0     0
2     10    6    90   43  686    6    14  33     1     0     0
3      3    5    91   33   77    9     8  97     4     0     0
4      3    7    89   51  102    9    11  99     1     0     0


In [ ]:
print("Statistical Description:", df.describe())

Statistical Description:             month         day        FFMC         DMC          DC         ISI  \
count  517.000000  517.000000  517.000000  517.000000  517.000000  517.000000   
mean     7.475822    4.259188   90.210832  110.487427  547.508704    8.603482   
std      2.275990    2.072929    5.526893   64.104372  248.108071    4.570658   
min      1.000000    1.000000   18.000000    1.000000    7.000000    0.000000   
25%      7.000000    2.000000   90.000000   68.000000  437.000000    6.000000   
50%      8.000000    5.000000   91.000000  108.000000  664.000000    8.000000   
75%      9.000000    6.000000   92.000000  142.000000  713.000000   10.000000   
max     12.000000    7.000000   96.000000  291.000000  860.000000   56.000000   

             temp          RH        wind        rain         area  
count  517.000000  517.000000  517.000000  517.000000   517.000000  
mean    18.417795   44.288201    3.557060    0.015474    12.599613  
std      5.797245   16.317469    1.797

In [ ]:
print("Shape:", df.shape)

Shape: (517, 11)


In [ ]:
print("Correlation:", df.corr(method='pearson'))

Correlation:           month       day      FFMC       DMC        DC       ISI      temp  \
month  1.000000 -0.050837  0.289814  0.466616  0.868639  0.189750  0.365026   
day   -0.050837  1.000000 -0.043008  0.063261  0.000135  0.019868  0.049511   
FFMC   0.289814 -0.043008  1.000000  0.381275  0.327776  0.535730  0.427536   
DMC    0.466616  0.063261  0.381275  1.000000  0.682180  0.313325  0.465960   
DC     0.868639  0.000135  0.327776  0.682180  1.000000  0.233980  0.491729   
ISI    0.189750  0.019868  0.535730  0.313325  0.233980  1.000000  0.399387   
temp   0.365026  0.049511  0.427536  0.465960  0.491729  0.399387  1.000000   
RH    -0.095280  0.092151 -0.297030  0.073746 -0.039150 -0.132676 -0.525514   
wind  -0.082907  0.018386 -0.013404 -0.098537 -0.197614  0.101231 -0.216514   
rain   0.013179 -0.034761  0.046998  0.060602  0.031654  0.055044  0.069908   
area   0.056080  0.023575  0.038401  0.072799  0.049063  0.004933  0.099051   

             RH      wind      rain   

In [ ]:
df.dtypes

month    int64
day      int64
FFMC     int64
DMC      int64
DC       int64
ISI      int64
temp     int64
RH       int64
wind     int64
rain     int64
area     int64
dtype: object

In [ ]:
data = df.values


X = data[:,0:10]
Y = data[:,10]

In [ ]:
len(data)

517

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesRegressor


In [ ]:
model = ExtraTreesRegressor()
rfe = RFE(model,step=3)
fit = rfe.fit(X, Y)

print("Number of Features: ", fit.n_features_)
print("Selected Features: ", fit.support_)
print("Feature Ranking: ", fit.ranking_)

Number of Features:  5
Selected Features:  [False  True False  True False False  True  True  True False]
Feature Ranking:  [3 1 3 1 2 2 1 1 1 3]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error

In [ ]:
num_instances = len(X)

models = []
models.append(('LiR', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('Lasso', Lasso()))
models.append(('RandomForest', RandomForestRegressor()))
models.append(('ExtraTreesRegressor', ExtraTreesRegressor()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('CART', DecisionTreeRegressor()))
models.append(('SVM', SVR()))

# Evaluations
results = []
names = []
scoring = []

for name, model in models:
    # Fit the model
    model.fit(X, Y)

    predictions = model.predict(X)

    # Evaluate the model
    score = explained_variance_score(Y, predictions)
    mae = mean_absolute_error(predictions, Y)
    # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    results.append(mae)
    names.append(name)

    msg = "%s: %f (%f)" % (name, score, mae)
    print(msg)



LiR: 0.019299 (19.234046)
Ridge: 0.019299 (19.232341)
Lasso: 0.018804 (19.049948)
RandomForest: 0.838366 (7.896967)
ExtraTreesRegressor: 0.986367 (1.042553)
KNN: 0.260535 (15.856093)
CART: 0.986367 (1.042553)
SVM: 0.000107 (12.600183)


In [ ]:
Y = numpy.array(Y).reshape((len(Y), 1))
#Y.reshape(-1, 1)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
Y = scaler.fit_transform(Y)

In [ ]:
from keras.layers import Dropout
from keras.utils import np_utils
from keras.constraints import maxnorm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [ ]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(10, input_dim=10, kernel_initializer='uniform', activation='relu'))
    #model.add(Dropout(0.2))
    #model.add(Dense(5, kernel_initializer='uniform', activation='relu', kernel_constraint=maxnorm(3)))
    #model.add(Dropout(0.2))
    model.add(Dense(3, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='relu'))

    # compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

    # evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=600, batch_size=10, verbose=0)

kfold = KFold(n_splits=30, random_state=None)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))


<ipython-input-138-a7a8996b79cd>:16: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=600, batch_size=10, verbose=0)


Results: -0.00 (0.01) MSE
